In [1]:
import osmnx as ox
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import nearest_points
import pyproj
from google.cloud import storage
from math import radians, sin, cos, sqrt, atan2


## Defining Functions

In [2]:
def haversine(lon1, lat1, lon2, lat2):
    # Convert degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = 6371 * c * 1000  # Earth radius in km, convert to meters
    return distance

def meters_to_degrees(meters, latitude):
    return meters / (111320 * cos(radians(latitude)))


## Extract Data using OSMNX
- Use OSMNX to get POI's within a certain radius of Austin city center 

In [3]:
# Define the latitude and longitude of the center point
latitude = 30.2645322996953
longitude = -97.7497791359203

# Define the radius in meters (25 km = 25000 meters)
radius = 35000

# Define amenities of interest
amenities_of_interest = {
    # food
    'cafe': ['coffee', 'green'],
    'bar': ['beer', 'green'],
    'restaurant': ['cutlery', 'green'],
    'pub': ['beer', 'green'],

    # social buildings
    'social_centre': ['institution', 'orange'],
    'library': ['book', 'orange'],
    'marketplace': ['shopping-cart', 'green'],
    'events_venue': ['institution', 'orange'],
    'exhibition_centre': ['institution', 'lightgrey'],
    'place_of_worship': ['institution', 'orange'],

    # places to chill
    'park': ['tree', 'green'],
    'music_venue': ['music', 'red'],
    'cinema': ['film', 'red'],
    'theatre': ['film', 'red']
}

# Define amenities of interest
amenities = list(amenities_of_interest.keys())
amenity_tags = {'amenity': amenities}

# Query the POIs within the specified radius
pois = ox.features.features_from_point((latitude, longitude), tags=amenity_tags, dist=radius)

# Filter the POIs to include only the amenities of interest
filtered_pois = pois[pois['amenity'].isin(amenities)]

# Create a new DataFrame with selected columns
columns_to_keep = ['amenity', 'name', 'geometry']  # Add other columns if needed
filtered_pois_df = filtered_pois[columns_to_keep]

# Add a column for the icon and color based on the amenities_of_interest dictionary
filtered_pois_df['icon'] = filtered_pois_df['amenity'].apply(lambda x: amenities_of_interest[x][0])
filtered_pois_df['color'] = filtered_pois_df['amenity'].apply(lambda x: amenities_of_interest[x][1])


c:\Users\Martin\anaconda3\envs\onebus\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\Martin\anaconda3\envs\onebus\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


#### Load Bus Stop Data

In [4]:
unique_bus_stops = pd.read_csv("all_unique_stops.csv")
unique_bus_stops

,Unnamed: 0,trip_id,arrival_time,departure_time,stop_sequence,trip_headsign,direction_id,route_id,wheelchair_accessible,bikes_allowed,stop_lat,stop_lon,stop_name,is_express
0,410083,2736572_13819,04:16:00,04:16:00,1,7 William Cannon SB,0,7,1,1,30.338721,-97.719548,7072 Easy Wind/St Johns,False
1,46760,2726327_0637,04:17:00,04:17:00,1,10 Southpark Meadows SB,0,10,1,1,30.340202,-97.691345,Norwood Transit Center - A,False
2,410084,2736572_13819,04:17:47,04:17:47,2,7 William Cannon SB,0,7,1,1,30.336202,-97.718247,6904 Airport/Lamar,False
3,46761,2726327_0637,04:18:28,04:18:28,2,10 Southpark Meadows SB,0,10,1,1,30.340454,-97.690058,1113 Rutherford/Furness,False
4,410085,2736572_13819,04:18:52,04:18:52,3,7 William Cannon SB,0,7,1,1,30.333155,-97.716673,Airport/Guadalupe,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3894,333432,2734188_8447,25:07:09,25:07:09,36,485 Downtown SB,1,485,1,1,30.268704,-97.727634,1200 11th/Lydia,False
3895,333433,2734188_8447,25:07:53,25:07:53,37,485 Downtown SB,1,485,1,1,30.269995,-97.731193,904 11th/Branch,False
3896,333434,2734188_8447,25:08:37,25:08:37,38,485 Downtown SB,1,485,1,1,30.271094,-97.734976,608 11th/Red River,False
3897,333435,2734188_8447,25:09:25,25:09:25,39,485 Downtown SB,1,485,1,1,30.272292,-97.739054,219 11th/San Jacinto,False


In [10]:
# Remove rows where the 'name' column is NaN
filtered_pois_df = filtered_pois_df.dropna(subset=['name','amenity'])

'''
General Logic:
1. Convert both bus stop and POI dataframe to geo dataframes
2. Bus Stop dataframe is converted to a "buffer" dataframe which is a polygon of N buffer distance given an x,y coordinate
3. Perform a spatial join between the POI and the bus stop buffer dataframe in order to get the POI accessible by bus stops

'''

# Step 1: Add a geometry column to the bus stop DataFrame
unique_bus_stops['geometry'] = unique_bus_stops.apply(lambda row: Point(row['stop_lon'], row['stop_lat']), axis=1)

# Step 2: Convert both DataFrames to GeoDataFrames
# Set the CRS to WGS84 (EPSG:4326) for latitude/longitude
bus_stops_gdf = gpd.GeoDataFrame(unique_bus_stops, geometry='geometry', crs='EPSG:4326')
pois_gdf = gpd.GeoDataFrame(filtered_pois_df, geometry='geometry', crs='EPSG:4326')

# Separate points and polygons
points_gdf = pois_gdf[pois_gdf.geometry.type == 'Point']
polygons_gdf = pois_gdf[pois_gdf.geometry.type == 'Polygon']

# Step 3: Create a copy for buffered bus stops
bus_stops_buffered_gdf = bus_stops_gdf.copy()

# Step 4: Calculate buffer distance in degrees

buffer_distance_degrees = meters_to_degrees(500, 30.26562)  # 500 meters at latitude 30.26562

# Step 5: Apply buffer to create circular areas around bus stops
bus_stops_buffered_gdf['geometry'] = bus_stops_buffered_gdf.geometry.buffer(buffer_distance_degrees)

# Step 6: Perform a spatial join to find POIs within the buffered area
pois_within_500m_points = gpd.sjoin(points_gdf,bus_stops_buffered_gdf,how='inner',predicate='within')
pois_within_500m_poly = gpd.sjoin(polygons_gdf, bus_stops_buffered_gdf, how='inner', predicate='intersects')

# Step 7: Calculate distance from centroid for polygons
pois_within_500m_poly['geometry'] = pois_within_500m_poly['geometry'].centroid

combined_poi = pd.concat([pois_within_500m_poly, pois_within_500m_points], ignore_index=True)


# Extract coordinates from geometry and stop_lat/stop_lon
combined_poi['distance'] = combined_poi.apply(
    lambda row: haversine(
        row.geometry.x, row.geometry.y,  # POI coordinates (lon, lat)
        row['stop_lon'], row['stop_lat']  # Bus stop coordinates
    ),
    axis=1
)

# Sort by distance (ascending) to prioritize shortest distances
combined_poi = combined_poi.sort_values('distance')

# Drop duplicates, keeping the closest POI per geometry
combined_poi_no_duplicates = combined_poi.drop_duplicates(subset=['geometry'], keep='first')

amenity_mapping = {
    'restaurant': 'Food Services',
    'marketplace': 'Commercial',
    'bar': 'Food Services',
    'cafe': 'Food Services',
    'place_of_worship': 'Community Services',
    'social_centre': 'Community Services',
    'events_venue': 'Recreation',
    'pub': 'Food Services',
    'library': 'Community Services',
    'cinema': 'Recreation',
    'theatre': 'Recreation',
    'music_venue': 'Recreation'
}

combined_poi_no_duplicates['amenity_category'] = combined_poi_no_duplicates['amenity'].map(amenity_mapping)

C:\Users\Martin\AppData\Local\Temp\ipykernel_124836\4189544917.py:32: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  bus_stops_buffered_gdf['geometry'] = bus_stops_buffered_gdf.geometry.buffer(buffer_distance_degrees)
C:\Users\Martin\AppData\Local\Temp\ipykernel_124836\4189544917.py:39: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  pois_within_500m_poly['geometry'] = pois_within_500m_poly['geometry'].centroid
c:\Users\Martin\anaconda3\envs\onebus\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

In [11]:
combined_poi_no_duplicates = combined_poi_no_duplicates[['amenity','amenity_category', 'name', 'geometry', 'icon', 'color']]

In [12]:
combined_poi_no_duplicates['amenity_category'].unique()

array(['Food Services', 'Commercial', 'Community Services', 'Recreation'],
      dtype=object)

## Data Loading to GCS

In [13]:
combined_poi_no_duplicates.to_csv('filtered_pois.csv', index=False)
